# Tecnicas para limpieza de datos

1. Eliminar duplicados
`drop_duplicates()`
2. Eliminar caracteres no deseados
`str.replace()`
3. Corrección del tipo de dato
df[ “campo” ].astype(float) [texto del enlace](https://)
4. Manejo de datos faltantes
`dropna()`
`fillna()`
5. Normalización de datos
`.str.strip().lower()`
6. Filtrado de datos
`df [ df[“campo”] + condición ]`


## Importamos algunos datasets para explorar

In [1]:
# Importar la librería Pandas
import pandas as pd

In [2]:
# Montar la unidad
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Verificar que los archivos csv se encuentren en la carpeta datasets
import os
os.listdir("/content/drive/MyDrive/datasets")

['netflix-shows.zip',
 'international-hotel-booking-analytics.zip',
 'netflix_titles.csv',
 'satis_clientes.csv',
 'sakila.db',
 'booking_db_nulls.sqlite',
 'booking_db.sqlite',
 'users.csv',
 'hotels.csv',
 'reviews.csv']

In [56]:
# Importamos el Dataset satis (customer satisfaction)
df_satis = pd.read_csv("/content/drive/MyDrive/datasets/satis_clientes.csv")
df_satis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1128 non-null   int64  
 1   Empresa       1128 non-null   object 
 2   Fecha         1128 non-null   object 
 3   Calificación  904 non-null    float64
 4   Comentarios   818 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 44.2+ KB


In [ ]:
# Importamos el Dataset Netflix
df_netflix = pd.read_csv("/content/drive/MyDrive/datasets/netflix_titles.csv")
df_netflix.info()

In [ ]:
# Importamos el Dataset de pacientes con sus mediciones de temperatura
df_pacientes = pd.read_csv('https://docs.google.com/spreadsheets/d/1-rUn4TUwpGrLE1DH8moeiR5eSyeF-jOOTpfhRgZxVIQ/gviz/tq?tqx=out:csv&sheet=')
df_pacientes.info()


## Identificación de datos duplicados

[duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html)

### Dataframe satis

In [ ]:
df_satis.info()

In [ ]:
# Veamos si hay duplicados
df_satis.duplicated().sum()

In [ ]:
df_satis.head(50)

In [ ]:
df_satis[["Empresa","Comentarios"]]

In [ ]:
# Veamos cuantos registros duplicados hay
df_satis.duplicated(subset=["Empresa", "Calificación"]).sum()

In [ ]:
# Contar valores unicos
df_satis.value_counts(subset=["Empresa", "Fecha"]).sort_values(ascending=False)

In [ ]:
# Visualizar los duplicados con diferentes argumentos
df_satis[df_satis.duplicated( keep=False)].sort_values(by='Empresa', ascending=True)

In [ ]:
# Aplicar filtros
df_satis[df_satis['Empresa'] == "Keebler Inc"]

In [ ]:
# Usamos slicing para filtrar columnas
df_satis.columns[1:]

In [ ]:
df_satis[df_satis.duplicated(subset=['Empresa', 'Fecha', 'Calificación', 'Comentarios'],keep=False)].sort_values(by='id')

In [ ]:
df_satis[df_satis.duplicated(subset=df_satis.columns[1:],keep=False)].sort_values(by='Empresa')

### Dataframe Pacientes

In [ ]:
# Veamos cuantos registros duplicados hay
df_pacientes.duplicated().sum()

In [ ]:
# Si hay, entonces los listamos (keep, first, last, False)
df_pacientes[df_pacientes.duplicated(subset=["nombre"], keep=False)].sort_values(by='nombre')

### Dataframe Netflix

In [ ]:
df_netflix.duplicated().sum()

In [ ]:
df_netflix[df_netflix.duplicated()]

## Tratamiento de datos duplicados
[Pandas drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

### Dataset Satis

In [ ]:
df_satis.info()

In [ ]:
df_satis.drop_duplicates(subset=["Empresa"], inplace=True, ignore_index=True) # pueden usar el inplace para sobrescribir el dataframe
df_satis_pp1 = df_satis.drop_duplicates(subset=["Empresa"], inplace=True, ignore_index=True) # O guardarlo en otra variable
df_satis.info()

In [ ]:
df_satis.head(50)

In [ ]:
df_satis_pp1[df_satis_pp1.duplicated(subset=["Empresa", "Calificación"], keep=False)].sort_values(by='Empresa')


In [ ]:
df_satis.duplicated().sum()

## Exploración de datos nulos

In [6]:
df_satis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1128 non-null   int64  
 1   Empresa       1128 non-null   object 
 2   Fecha         1128 non-null   object 
 3   Calificación  904 non-null    float64
 4   Comentarios   818 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 44.2+ KB


In [ ]:
# Ver si hay valores nulos, en general o en columnas especificas
df_satis.isnull().sum()
#df_satis[["Calificación"]].isnull().sum()
#df_satis.isnull().any(axis=1).sum()

In [ ]:
# Ambos nos muestras la cantidad de nulos por columna
df_satis[df_satis.columns].isnull().sum()
df_satis.isnull().sum()

In [ ]:
# Visualizar los registros con celdas null / NaN
df_satis[df_satis.isnull().any(axis=1)]

## Tratamiento de datos nulos

### drop

`dropna` se usa para eliminar registros que contengan datos nulos
<BR>
[Pandas dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)

* how: para especificar si (any / all) celdas deben tener NaN
* thresh: indicar la cantidad de NaN para ejecutar el drop (no se puede combinar con any)
* subset: indicar las columnas a evaluar
* inplace: si guarda los cambios en el dataframe o retorna una copia

Analizar y comparar que sucede cuando aplicamos dropna a todas las columnas o a algunas especificas.

In [ ]:
# Eliminar un registro si alguna las columnas contienen NaA
df_satis_pp2 = df_satis.dropna()
df_satis_pp2.info()

In [ ]:
# Eliminar un registro si alguna o tdas las columnas contienen NaA
df_satis_pp2 = df_satis.dropna(how="all")
df_satis_pp2.info()

In [ ]:
# Eliminar un registro solo evaluando NaN en las columnas indicadas
df_satis_pp2 = df_satis.dropna(subset=["Calificación"])
df_satis_pp2.info()

In [ ]:
# Validamos que no haya celdas con Null luego del Drop
df_satis_pp2[df_satis.isnull().any(axis=1)]

In [ ]:
# Eliminar una columna si contiene NaN
df_satis_pp2 = df_satis.dropna(axis=1) # no accepta subset
df_satis_pp2.info()

### fill

`fillna` se utiliza para completar el dato faltante
<BR>
[Pandas fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)
<BR>
* df.fillna(0)  se completa con un valor fijo
* df.fillna(method="ffill")  completa con el valor del registro anterior
* df.fillna(method="bfill") completa con el valor del registro previo
* df["col"].fillna(df["col"].mean()) completa con la media de la columna
* df.fillna({"col1": 0, "col2": "desconocido"})

1. completamos con un valor fijo

In [35]:
# Visualicemos algunos NaN para ver que sucede luego del tratamiento
# df_satis[df_satis.isnull().any(axis=1)]
# df_satis.loc[3:9, ["Calificación", "Comentarios"]]
df_satis_pp3.loc[3:8, ["Calificación", "Comentarios"]]

,Calificación,Comentarios
3,1.0,Integer ac leo. Pellentesque ultrices mattis o...
4,4.0,Integer ac leo. Pellentesque ultrices mattis o...
5,1.0,Phasellus in felis. Donec semper sapien a libe...
6,5.0,"Donec diam neque, vestibulum eget, vulputate u..."
7,5.0,"Donec diam neque, vestibulum eget, vulputate u..."
8,5.0,Nullam sit amet turpis elementum ligula vehicu...


In [ ]:
# usamos el atributo value con un valor fijo en 0
df_satis_pp3 = df_satis.fillna(value = 0)
df_satis_pp3.info()

In [27]:
# También podríamos usar un diccionario
df_satis_pp3 = df_satis.fillna({"Calificación": 0, "Comentarios": "Sin dato"})

2. Completamos con el valor previo

In [ ]:
df_satis_pp3 = df_satis.fillna(method = "ffill")
df_satis_pp3.info()

3. Completamos con el valor próximo

In [ ]:
df_satis_pp3 = df_satis.fillna(method = "bfill")
df_satis_pp3.info()

4. Completamos con la media, meadiana, moda

In [36]:
# Primero calculamos la media/mediana/moda y luego aplicamos
media_calificacion = df_satis["Calificación"].mean()
df_satis_pp3 = df_satis.fillna(media_calificacion)
df_satis_pp3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1128 non-null   int64  
 1   Empresa       1128 non-null   object 
 2   Fecha         1128 non-null   object 
 3   Calificación  1128 non-null   float64
 4   Comentarios   1128 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 44.2+ KB


In [ ]:
# O directamente, usan el numeric only
df_satis_pp3 = df_satis.fillna(df_satis.mean(numeric_only=True))
df_satis_pp3.info()

In [ ]:
# aplicar fillna con diccionario
df_satis_pp3 = df_satis.fillna({
    "Calificación": media_calificacion,
    "Comentarios": "Sin dato"
})

Analicemos el dataset pacientes

In [37]:
df_pacientes.info()

NameError: name 'df_pacientes' is not defined

In [ ]:
# Completamos con la media de la columna
df_pacientes_pp1 = df_pacientes.fillna(df_pacientes.mean(numeric_only=True))
df_pacientes_pp1.info()

In [ ]:
# Completamos con la media de la fila usando lambda

# seleccionar solo las columnas d1...d10
cols = df_pacientes.columns[1:]   # todas excepto 'nombre'

# aplicar moda por fila
df_pacientes_pp1[cols] = df_pacientes[cols].apply(
    lambda row: row.fillna(row.mean()), axis=1
)

In [ ]:
# Completamos con la media de la fila usando T (traspuesta)
df_pacientes_pp1[cols] = df_pacientes[cols].T.fillna(df_pacientes[cols].mean(axis=1)).T

In [ ]:
#df_pacientes.head()
df_pacientes_pp1.head()

Veamos el concepto de Traspuesta

In [ ]:
df = pd.DataFrame({
    "A": [1, 2, 3],
    "B": [4, 5, 6]
})
print("Original:")
print(df)

print("\nTranspuesta:")
print(df.T)

## Normalización de datos

In [57]:
df_satis.info()
df_satis.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1128 non-null   int64  
 1   Empresa       1128 non-null   object 
 2   Fecha         1128 non-null   object 
 3   Calificación  904 non-null    float64
 4   Comentarios   818 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 44.2+ KB


,id,Empresa,Fecha,Calificación,Comentarios
0,1,Mitchell Group,11/12/2024,1.0,Integer ac leo. Pellentesque ultrices mattis o...
1,2,Kuhn-Fay,25/01/2024,4.0,Vestibulum ac est lacinia nisi venenatis trist...
2,3,Moen-Blick,11/11/2024,3.0,Aliquam quis turpis eget elit sodales sceleris...
3,4,McDermott Inc,01/12/2024,1.0,NaN
4,5,Keebler Inc,12/01/2024,4.0,Integer ac leo. Pellentesque ultrices mattis o...


In [ ]:
# Eliminiar espacios en blanco
df_satis["Empresa"] = df_satis["Empresa"].str.strip()

In [40]:
# Cambiar mayúsculas/minúsculas
df_satis["Empresa"] = df_satis["Empresa"].str.lower()

In [65]:
# Reemplazar caracteres
df_satis["Empresa_norm"] = df_satis["Empresa"].str.replace("Mitchell Group", "Grupo Mitchell")

In [69]:
# Con regex
df_satis["Empresa_norm"] = df_satis["Empresa"].str.replace(r"\bInc\b", "inc.", regex=True)

In [ ]:
df_satis["Empresa_norm"]

## Ajustar tipo de dato

### astype()

* df["col"] = df["col"].astype(int)
* df["col"] = df["col"].astype(float)
* df["col"] = df["col"].astype(str)

In [86]:
df_satis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1128 non-null   Int64  
 1   Empresa       1128 non-null   object 
 2   Fecha         1128 non-null   object 
 3   Calificación  904 non-null    float64
 4   Comentarios   818 non-null    object 
 5   Empresa_norm  1128 non-null   object 
dtypes: Int64(1), float64(1), object(4)
memory usage: 54.1+ KB


In [ ]:
df_satis_pp4 = df_satis.copy()
df_satis_pp4["Comentarios"] = df_satis["Comentarios"].astype("string")

In [ ]:
df_satis_pp4.info()

### to_numeric()

In [83]:
# Como no tenemos una columna con números en formato str lo forzamos
df_satis["id"] = df_satis["id"].astype("string")

In [85]:
# Y luego usamos pd.to_numeric para convertir a int
df_satis["id"] = pd.to_numeric(df_satis["id"], errors="coerce")

### to_datetime()

In [87]:
# Veamos el tipo de dato del campo "Fecha"
df_satis["Fecha"].dtypes

dtype('O')

In [93]:
# Imprimos algun valor
df_satis["Fecha"][0]
df_satis.loc[:5, "Fecha"]

,Fecha
0,2024-12-11
1,2024-01-25
2,2024-11-11
3,2024-12-01
4,2024-01-12
5,2024-04-30


In [92]:
df_satis["Fecha"] = pd.to_datetime(df_satis["Fecha"], format="%d/%m/%Y")

In [ ]:
# pd.to_datetime(df_satis.Fecha)
pd.to_datetime(df_satis["Fecha"], format="%d/%m/%Y")


In [ ]:
# Ahora podemos acceder al year/month/day
df_satis["Fecha"].dt.year
# df_satis["Fecha"].dt.month
# df_satis["Fecha"].dt.day

In [96]:
# Ahora podemos acceder al year/month/day
df_satis["Fecha"][0].year

2024